In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt

# Chargement des données (à adapter pour chaque sujet)
raw = mne.io.read_raw_edf("data/S001R01.edf", preload=True)
raw.set_eeg_reference('average', projection=True)

# Appliquer montage standard
montage = mne.channels.make_standard_montage("standard_1020")
raw.set_montage(montage, on_missing="ignore")

# Supprimer les canaux conflictuels
bad_chs = ['FCZ', 'CZ', 'CPZ', 'FP1', 'FPZ', 'FP2', 'AFZ', 'FZ', 'PZ', 'POZ', 'OZ', 'IZ']
raw.pick_channels([ch for ch in raw.ch_names if ch not in bad_chs])

# Filtrage passe-bande classique EEG
raw.filter(1., 40., fir_design='firwin')

# Découpage en epochs autour des événements
events, _ = mne.events_from_annotations(raw)
event_id = dict(rest=1, left=2, right=3)  # Adapter selon annotations

epochs = mne.Epochs(raw, events, event_id=event_id, tmin=0, tmax=2,
                    baseline=None, preload=True, detrend=1)

# Normalisation par z-score
data = epochs.get_data()  # shape (n_trials, n_channels, n_times)
data = (data - data.mean(axis=-1, keepdims=True)) / data.std(axis=-1, keepdims=True)

# Reshape pour EEGNet: [N, 1, C, T]
X = data[:, np.newaxis, :, :]
y = epochs.events[:, 2]

print(f"✅ Données préparées : X shape = {X.shape}, y shape = {y.shape}")
np.save("X_windows.npy", X)
np.save("y_windows.npy", y)


Extracting EDF parameters from C:\Users\Antoi\Documents\EEG-BCI\notebooks\data\S001R01.edf...
EDF file detected


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Antoi\\Documents\\EEG-BCI\\notebooks\\data\\S001R01.edf'